In [2]:
from IPython.display import Image
import ee
import ee.mapclient
import datetime

ee.Initialize()
L1_AGBPSeasonals = ee.ImageCollection("projects/fao-wapor/L1_AGBP")
ETaCollection = ee.ImageCollection("projects/fao-wapor/L1_AET")
AET250 = ee.ImageCollection("users/lpeiserfao/AET250")

rasterSeason1 = ee.Image(L1_AGBPSeasonals.first())

# Informazioni varie sui dati di input

In [3]:
rasterSeason1.getInfo()

{u'bands': [{u'crs': u'EPSG:4326',
   u'crs_transform': [0.00223214286,
    0.0,
    -30.0044643,
    0.0,
    -0.00223214286,
    40.0044643],
   u'data_type': {u'precision': u'float', u'type': u'PixelType'},
   u'dimensions': [42564, 35844],
   u'id': u'b1'}],
 u'id': u'projects/fao-wapor/L1_AGBP/L1_AGBP_15s1',
 u'properties': {u'area': u'AfNE',
  u'days_in_dk': 0.0,
  u'season': 1.0,
  u'system:asset_size': 1602581313,
  u'system:footprint': {u'coordinates': [[33.45815610768672,
     40.00558428068579],
    [23.809038489303575, 40.00558428068579],
    [9.335362061737218, 40.00558428068581],
    [-1.0559953734367642, 40.005584280685795],
    [-14.78743198420065, 40.005584280685795],
    [-30.00626732411657, 40.005579588766935],
    [-30.00626732411973, -40.00557966260693],
    [-18.127511159796093, -40.00558435452578],
    [0.057364351758189706, -40.00558435452578],
    [13.788800962527548, -40.00558435452578],
    [33.82927601608015, -40.00558435452579],
    [51.27191171084418, -40.

In [4]:
coordinate = rasterSeason1.get('system:footprint')
coordinate.getInfo()

{u'coordinates': [[33.45815610768672, 40.00558428068579],
  [23.809038489303575, 40.00558428068579],
  [9.335362061737218, 40.00558428068581],
  [-1.0559953734367642, 40.005584280685795],
  [-14.78743198420065, 40.005584280685795],
  [-30.00626732411657, 40.005579588766935],
  [-30.00626732411973, -40.00557966260693],
  [-18.127511159796093, -40.00558435452578],
  [0.057364351758189706, -40.00558435452578],
  [13.788800962527548, -40.00558435452578],
  [33.82927601608015, -40.00558435452579],
  [51.27191171084418, -40.00558435452578],
  [65.0062674171597, -40.00557966260692],
  [65.00626741715656, 40.00557958876695],
  [54.240870978035474, 40.005584280685795],
  [44.220633451257086, 40.00558428068579],
  [33.45815610768672, 40.00558428068579]],
 u'type': u'LinearRing'}

In [5]:
illo = rasterSeason1.get('system:index'),2
type(illo)

tuple

In [6]:
index = rasterSeason1.get('system:index')
index.getInfo()

u'L1_AGBP_15s1'

In [7]:
bande = rasterSeason1.bandNames()
bande.getInfo()

[u'b1']

In [8]:
region = [[-25.0, -37.0], [60.0, -41.0], [58.0, 39.0], [-31.0, 38.0], [-25.0, -37.0]]

# Inizio calcolo WP

### Above Ground Biomass Production with masked NoData (pixel < 0) and summed

In [9]:
L1_AGBPSeasonalMasked =L1_AGBPSeasonals.map(lambda lista: lista.updateMask(lista.gte(0)))

#the multiplier will need to be applied on net FRAME delivery, not on sample dataset
L1_AGBPSummedYearly = L1_AGBPSeasonalMasked.sum(); #.multiply(10); 

### Actual Evapotranspiration with valid ETa values (>0 and <254)

In [10]:
ETaColl1 = AET250.map(lambda immagine: immagine.updateMask(immagine.lt(254) and (immagine.gt(0)))) 

### add image property (days in dekad) as band

In [11]:
ETaColl2 = ETaColl1.map(lambda immagine: immagine.addBands(immagine.metadata('days_in_dk')));

### get ET value, divide by 10 (as per FRAME spec) to get daily value, and multiply by number of days in dekad summed annually

In [12]:
ETaColl3 = ETaColl2.map(lambda immagine: immagine.select('b1').divide(10).multiply(immagine.select('days_in_dk'))).sum()

### scale ETsum from mm/m² to m³/ha for WP calculation purposes

In [13]:
ETaTotm3 = ETaColl3.multiply(10)

### calculate biomass water productivity and add to map

In [23]:
WPbm = L1_AGBPSummedYearly.divide(ETaTotm3)
WPbm.getInfo()

{u'bands': [{u'crs': u'EPSG:4326',
   u'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   u'data_type': {u'precision': u'double', u'type': u'PixelType'},
   u'id': u'b1'}],
 u'type': u'Image'}

In [36]:
region = [[-25.0, -37.0], [60.0, -41.0], [58.0, 39.0], [-31.0, 38.0], [-25.0, -37.0]]
scala = WPbm.projection().nominalScale().getInfo()

visparams = {"opacity":1,
             "min":0,
             "max":1.2,
             "palette":"bc170f,e97a1a,fff83a,9bff40,5cb326",
             "region":region,
             "scale" : scala}

In [37]:
url_WPbm = WPbm.getThumbUrl(visparams)
Image(url=url_WPbm)

In [29]:
downConfig = {'scale': scala,
              'maxPixels' : 1500000000,
              'region': region};

fileName = 'WP_py'
export = ee.batch.Export.image(WPbm, fileName,downConfig);
#export.start();